In [1]:
from mypulp import *
import numpy as np
import itertools
import networkx as nx
import math
import matplotlib.pyplot as plt
%matplotlib inline

# スケジューリング問題

## 1機械リリース時刻重み付き完了時刻和問題

In [14]:
def scheduling_disjunctive(J, p, r, w):
    model = Model('scheduling_disjunctive')
    M = max(r) + sum(p)

    ## 変数 ##
    s, x = {}, {}
    for j in J:
        s[j] = model.addVar(lb=r[j], vtype='C')
        for k in J:
            if j != k:
                x[j, k] = model.addVar(vtype='B', name=f'x_{j},{k}')
    model.update()
    
    ## 定式化 ##
    for j in J:
        for k in J:
            if j != k:
                model.addConstr(s[j] - s[k] + M*x[j, k] <= (M - p[j]))
            if j < k:
                model.addConstr(x[j, k] + x[k, j] == 1)
    model.setObjective(quicksum(w[j] * s[j] for j in J), GRB.MINIMIZE)
    
    model.update()
    model.__data = s, x
    
    return model

In [17]:
p = [1, 4, 2, 3, 1, 4]
r = [4, 0, 2, 4, 1, 5]
w = [3, 1, 2, 3, 1, 2]
J = range(len(p))

model = scheduling_disjunctive(J, p, r, w)
model.optimize()
s, x = model.__data

# 最適解
print(f'Optimal Value: {model.ObjVal}')

eps = 1.0e-6
optsol = []
for j, k in x:
    if x[j, k].X > eps:
        optsol.append(x[j, k])
print(f'Optimal Solution: {optsol}')

Optimal Value: 60.0
Optimal Solution: [x_0,1, x_0,3, x_0,5, x_2,0, x_2,1, x_2,3, x_2,5, x_3,1, x_3,5, x_4,0, x_4,1, x_4,2, x_4,3, x_4,5, x_5,1]
